### Create Environmental Exclusion Layer
In this notebook we create an environmental exclusion layer for utility scale solar or onshore wind installations in the CONUS. We will use this layer to mask our 'Techno Economic Inclusion Layer' to produce the SL1 Inclusion Area Layer and subsequent area assessments. 

In [17]:
import arcpy
import os
from arcpy import env
from arcpy.sa import *

#### Define some useful functions

In [18]:
# grab all feature classes from a folder or geodatabase
def get_feature_classes(workspace):
    # Get the current workspace
    previous_workspace = arcpy.env.workspace

    try:
        # Set the new workspace
        arcpy.env.workspace = workspace

        # Get a list of all feature classes in the workspace
        feature_classes = arcpy.ListFeatureClasses()
        feature_classes = [os.path.join(arcpy.env.workspace, r) for r in feature_classes]

        # Return the list of feature classes
        return feature_classes

    finally:
        # Reset the workspace to the previous workspace
        arcpy.env.workspace = previous_workspace

In [37]:
# grab all rasters from a folder or geodatabase
def get_rasters(workspace):
    # Get the current workspace
    previous_workspace = arcpy.env.workspace

    try:
        # Set the new workspace
        arcpy.env.workspace = workspace

        # Get a list of all feature classes in the workspace
        raster_names = arcpy.ListRasters()
        rasters = [os.path.join(arcpy.env.workspace, r) for r in raster_names]

        # Return the list of rasters
        return rasters

    finally:
        # Reset the workspace to the previous workspace
        arcpy.env.workspace = previous_workspace

In [19]:
def get_datum_transformation(source_spatial_reference, target_spatial_reference):
    transformations = arcpy.ListTransformations(source_spatial_reference, target_spatial_reference)
    if transformations:
        return transformations[0]  # Use the first transformation found
    else:
        return None  # No appropriate transformation found

In [20]:
def reproject_feature_class(input_fc, template_sr):
    # Get the spatial reference of the input fc
    input_fc_sr = arcpy.Describe(input_fc).spatialReference

    # Check if the input fc spatial reference is different from the template spatial reference
    if input_fc_sr.exportToString() != template_sr.exportToString():
        print(f"Reprojecting {input_fc} to match the template raster's coordinate system...")
        input_fc_name = os.path.basename(input_fc)
        reprojected_fc = os.path.join(reprojected_geodatabase, f"{input_fc_name}_reprojected")

        # Get best datum transformation
        datum_transform = get_datum_transformation(input_fc_sr, template_sr)

        # Reproject feature class with appropriate datum transformation
        arcpy.management.Project(input_fc, reprojected_fc, template_sr, datum_transform)

        # Return the reprojected feature class path
        return reprojected_fc

    # Return the input feature class path to indicate it does not require reprojection
    return input_fc

In [21]:
def add_occupancy_field(feature_class):
    # Check if the 'occupancy' field already exists
    fields = arcpy.ListFields(feature_class, "occupancy")

    if not fields:
         # Add 'occupancy' field to the feature class if it doesn't exist
        arcpy.management.AddField(feature_class, "occupancy", "SHORT")

        # Set the 'occupancy' field value to 1 for all records
        with arcpy.da.UpdateCursor(feature_class, "occupancy") as cursor:
            for row in cursor:
                row[0] = 1
                cursor.updateRow(row)
        print(f"'occupancy' field added and set to 1 in {os.path.basename(feature_class)}")
    else:
        print(f"'occupancy' field already exists in {os.path.basename(feature_class)}")

#### Set input and output paths, set environmental variables for raster analysis

In [22]:
# set the path to the main input folder
mainInputFolder = "C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data"

In [23]:
# Set the workspace for the geodatabase containing the input feature classes
arcpy.env.workspace = os.path.join(mainInputFolder, "environmentalExclusionSolar.gdb")

In [24]:
# set the path to the geodatabase containing reprojected feature classes
reprojected_geodatabase_name = "environmentalExclusionSolarReprojected.gdb"
reprojected_geodatabase = os.path.join(mainInputFolder, reprojected_geodatabase_name)

if not arcpy.Exists(reprojected_geodatabase):
    try:
        arcpy.management.CreateFileGDB(mainInputFolder, reprojected_geodatabase_name)
        print(f"File geodatabase '{reprojected_geodatabase_name}.gdb' created successfully.")
    except:
        print(f"Error creating file geodatabase: {Exception}")

File geodatabase 'environmentalExclusionSolarReprojected.gdb.gdb' created successfully.


In [25]:
# set path to output folder
output_workspace = os.path.join(mainInputFolder, "environmentalExclusionRasters\\SolarScratch")

if not os.path.exists(output_workspace):
    os.makedirs(output_workspace)

In [40]:
# set path to mosaic output folder
mosaic_workspace = os.path.join(mainInputFolder, "environmentalExclusionRasters\\mosaicScratch")

if not os.path.exists(mosaic_workspace):
    os.makedirs(mosaic_workspace)

In [26]:
# set the path to the template raster
template_raster = "C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\SRTM_250m_proj_cl.tif"

In [27]:
# set environments for raster analyses
arcpy.env.snapRaster = template_raster
arcpy.env.extent = template_raster
arcpy.env.mask = template_raster
arcpy.env.cellSize = template_raster

#### Reproject input feature classes to the same spatial reference as the template raster

In [28]:
# Get the spatial reference of the template raster
template_srid = arcpy.Describe(template_raster).spatialReference
template_srid

name (Projected Coordinate System),USA_Contiguous_Albers_Equal_Area_Conic_USGS_version
factoryCode (WKID),102039
linearUnitName (Linear Unit),Meter
name (Geographic Coordinate System),GCS_North_American_1983
factoryCode (WKID),4269
angularUnitName (Angular Unit),Degree
datumName (Datum),D_North_American_1983


In [29]:
# get the list of feature classes in the geodatabase
feature_classes = get_feature_classes(arcpy.env.workspace)
feature_classes

['C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\environmentalExclusionSolar.gdb\\BLM_DRECP_NCL',
 'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\environmentalExclusionSolar.gdb\\BLM_DRECP_SRMA',
 'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\environmentalExclusionSolar.gdb\\BLM_National_Conservation_Areas',
 'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\environmentalExclusionSolar.gdb\\BLM_National_Scenic_Trails_Buffer500',
 'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\environmentalExclusionSolar.gdb\\BLM_Right_Of_Way_Exclusions',
 'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\environmentalExclusionSolar.gdb\\BLM_Solar_Energy_Program_Exclusions',
 'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\environmentalExclusionSolar.gdb\\BLM_Visual_Resource_Management_Class_I',
 'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\environmentalExclusionSolar.gdb\\BLM_Visual_Resource_Management_Class_II',
 'C:\\Users\\Za

In [30]:
print(arcpy.Describe(feature_classes[0]).spatialReference.exportToString())

PROJCS["USA_Contiguous_Albers_Equal_Area_Conic_USGS_version",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-96.0],PARAMETER["Standard_Parallel_1",29.5],PARAMETER["Standard_Parallel_2",45.5],PARAMETER["Latitude_Of_Origin",23.0],UNIT["Meter",1.0]];-16901100 -6972200 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision


In [31]:
print(arcpy.Describe(template_raster).spatialReference.exportToString())

PROJCS["USA_Contiguous_Albers_Equal_Area_Conic_USGS_version",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-96.0],PARAMETER["Standard_Parallel_1",29.5],PARAMETER["Standard_Parallel_2",45.5],PARAMETER["Latitude_Of_Origin",23.0],UNIT["Meter",1.0]];-16901100 -6972200 266467840.990852;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision


In [32]:
for feature_class in feature_classes:
    reprojected_fc = reproject_feature_class(feature_class, template_srid)

    if reprojected_fc is None:
        copied_fc = os.path.join(reprojected_geodatabase, os.path.basename(feature_class))
        arcpy.management.CopyFeatures(feature_class, copied_fc)

print("All feature classes have been processed and saved to the output geodatabase.")


Reprojecting C:\Users\Zachary\ASSET\supplyCurve\analysis\data\environmentalExclusionSolar.gdb\BLM_DRECP_NCL to match the template raster's coordinate system...
Reprojecting C:\Users\Zachary\ASSET\supplyCurve\analysis\data\environmentalExclusionSolar.gdb\BLM_DRECP_SRMA to match the template raster's coordinate system...
Reprojecting C:\Users\Zachary\ASSET\supplyCurve\analysis\data\environmentalExclusionSolar.gdb\BLM_National_Conservation_Areas to match the template raster's coordinate system...
Reprojecting C:\Users\Zachary\ASSET\supplyCurve\analysis\data\environmentalExclusionSolar.gdb\BLM_National_Scenic_Trails_Buffer500 to match the template raster's coordinate system...
Reprojecting C:\Users\Zachary\ASSET\supplyCurve\analysis\data\environmentalExclusionSolar.gdb\BLM_Right_Of_Way_Exclusions to match the template raster's coordinate system...
Reprojecting C:\Users\Zachary\ASSET\supplyCurve\analysis\data\environmentalExclusionSolar.gdb\BLM_Solar_Energy_Program_Exclusions to match the t

#### Convert reprojected feature classes to rasters

In [33]:
# Get all reprojected feature classes
feature_classes  = get_feature_classes(reprojected_geodatabase)

In [34]:
# Add occupancy field if doesn't exist
for feature_class in feature_classes:
    add_occupancy_field(feature_class)

'occupancy' field already exists in BLM_DRECP_NCL_reprojected
'occupancy' field already exists in BLM_DRECP_SRMA_reprojected
'occupancy' field already exists in BLM_National_Conservation_Areas_reprojected
'occupancy' field already exists in BLM_National_Scenic_Trails_Buffer500_reprojected
'occupancy' field already exists in BLM_Right_Of_Way_Exclusions_reprojected
'occupancy' field already exists in BLM_Solar_Energy_Program_Exclusions_reprojected
'occupancy' field already exists in BLM_Visual_Resource_Management_Class_I_reprojected
'occupancy' field already exists in BLM_Visual_Resource_Management_Class_II_reprojected
'occupancy' field already exists in Conservation_Mitigation_Bank_reprojected
'occupancy' field already exists in Historic_Cultural_Area_reprojected
'occupancy' field already exists in Marine_Protected_Areas_reprojected
'occupancy' field already exists in National_Conservation_Areas_reprojected
'occupancy' field already exists in National_Lakeshore_Seashore_reprojected
'occ

In [36]:
for feature_class in feature_classes:
    # get name of feature class
    feature_class_name = os.path.basename(feature_class)

    # convert each feature class to a raster and save to raster output folder
    output_raster = os.path.join(output_workspace, f"{feature_class_name}.tif")
    arcpy.conversion.FeatureToRaster(feature_class, "occupancy", output_raster)

    print(f"Feature class {feature_class_name} converted to raster and saved as {feature_class_name}.tif")

#### Mosaic the rasters to a new raster

In [39]:
# List the rasters in the raster folder
rasters = get_rasters(output_workspace)
rasters

['C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\environmentalExclusionRasters\\SolarScratch\\BLM_DRECP_NCL_reprojected.tif',
 'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\environmentalExclusionRasters\\SolarScratch\\BLM_DRECP_SRMA_reprojected.tif',
 'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\environmentalExclusionRasters\\SolarScratch\\BLM_National_Conservation_Areas_reprojected.tif',
 'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\environmentalExclusionRasters\\SolarScratch\\BLM_National_Scenic_Trails_Buffer500_reprojected.tif',
 'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\environmentalExclusionRasters\\SolarScratch\\BLM_Right_Of_Way_Exclusions_reprojected.tif',
 'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\environmentalExclusionRasters\\SolarScratch\\BLM_Solar_Energy_Program_Exclusions_reprojected.tif',
 'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\environmentalExclusionRasters\\SolarScratch\\BLM_Visua

In [42]:
# Mosaic all the rasters into one raster
mosaic_name = "environementalExclusionSolar_SL1_ZRM.tif"
arcpy.management.MosaicToNewRaster(input_rasters = rasters, 
                                   output_location = mosaic_workspace, 
                                   raster_dataset_name_with_extension=mosaic_name,
                                   number_of_bands=1,
                                   mosaic_method='FIRST')

<Result 'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\environmentalExclusionRasters\\mosaicScratch\\environementalExclusionSolar_SL1_ZRM.tif'>